<a href="https://colab.research.google.com/github/severancej/colorful_project/blob/main/AI_14_%EC%9E%A5%EC%A7%80%EC%9D%80_Section1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# <항상 시작 전>라이브러리 불러오기

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
#파일 업로드
from google.colab import files
uploaded = files.upload() 

Saving vgames2.csv to vgames2.csv


In [47]:
# .read_csv() 활용하여 데이터 불러오기
df = pd.read_csv('vgames2.csv')

#데이터 정제 전 원본copy함
df1 = df.copy()

In [48]:

#데이터셋의 shape 확인
print(df1.shape)
print("")
print("")

#데이터셋의 info 확인
print(df1.info( ))


(16598, 10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   16598 non-null  int64  
 1   Name         16598 non-null  object 
 2   Platform     16598 non-null  object 
 3   Year         16327 non-null  float64
 4   Genre        16548 non-null  object 
 5   Publisher    16540 non-null  object 
 6   NA_Sales     16598 non-null  object 
 7   EU_Sales     16598 non-null  object 
 8   JP_Sales     16598 non-null  object 
 9   Other_Sales  16598 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 1.3+ MB
None


In [49]:
#🕹️ Data Description
#Name : 게임의 이름입니다.
#Platform : 게임이 지원되는 플랫폼의 이름입니다.
#Year : 게임이 출시된 연도입니다.
#Genre : 게임의 장르입니다.
#Publisher : 게임을 제작한 회사입니다.
#NA_Sales : 북미지역에서의 출고량입니다.
#EU_Sales : 유럽지역에서의 출고량입니다.
#JP_Sales : 일본지역에서의 출고량입니다.
#Other_Sales : 기타지역에서의 출고량입니다.
df1.head( )

Unnamed: 0                                            Name Platform  \
0           1                    Candace Kane's Candy Factory       DS   
1           2                                  The Munchables      Wii   
2           3  Otome wa Oanesama Boku ni Koi Shiteru Portable      PSP   
3           4                Deal or No Deal: Special Edition       DS   
4           5       Ben 10 Ultimate Alien: Cosmic Destruction      PS3   

     Year      Genre           Publisher NA_Sales EU_Sales JP_Sales  \
0  2008.0     Action           Destineer     0.04        0        0   
1  2009.0     Action  Namco Bandai Games     0.17        0        0   
2  2010.0  Adventure           Alchemist        0        0     0.02   
3  2010.0       Misc           Zoo Games     0.04        0        0   
4  2010.0   Platform         D3Publisher     0.12     0.09        0   

  Other_Sales  
0           0  
1        0.01  
2           0  
3           0  
4        0.04

In [50]:
# 데이터의 결측치 확인 - df1
pd.DataFrame(df1.isnull().sum(), columns=["결측치 개수"])

결측치 개수
Unnamed: 0        0
Name              0
Platform          0
Year            271
Genre            50
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0

In [66]:
df1['Year'].value_counts( )


2009.0    1421
2008.0    1418
2010.0    1252
2007.0    1199
2011.0    1129
          ... 
13.0         2
12.0         2
86.0         1
2020.0       1
94.0         1
Name: Year, Length: 62, dtype: int64

In [63]:
df1['Genre'].value_counts( )

Action          3305
Sports          2341
Misc            1734
Role-Playing    1483
Shooter         1308
Adventure       1280
Racing          1243
Platform         884
Simulation       865
Fighting         847
Strategy         680
Puzzle           578
Name: Genre, dtype: int64

In [55]:
# 데이터 중 Year, Genre, Publisher 컬럼의 결측값 확인
# df1[df1['Year'].isnull() | df1['Genre'].isnull() | df1['Publisher'].isnull()]
df1[df1['Year'].isnull()]

Unnamed: 0                                         Name Platform  Year  \
31             32                                    Rock Band     X360   NaN   
109           110   Yu-Gi-Oh! 5D's Wheelie Breakers (JP sales)      Wii   NaN   
273           274  Jewel Link Chronicles: Mountains of Madness       DS   NaN   
358           359           Majesty 2: The Fantasy Kingdom Sim     X360   NaN   
430           431                                   Famista 64      N64   NaN   
...           ...                                          ...      ...   ...   
16240       16241                                Freaky Flyers       GC   NaN   
16275       16276                               Space Invaders     2600   NaN   
16278       16279                                Madden NFL 11      Wii   NaN   
16425       16426                               Yoostar on MTV     X360   NaN   
16432       16433                                  Bejeweled 3     X360   NaN   

            Genre           Publisher NA_Sales EU_Sales JP_Sales Other_Sales  
31           Misc     Electronic Arts     1.93     0.34        0        0.21  
109        Racing             Unknown        0        0     0.02           0  
273        Puzzle           Avanquest        0     0.06        0        0.01  
358    Simulation             Unknown     0.03        0        0           0  
430        Sports  Namco Bandai Games        0        0     0.17        0.03  
...           ...                 ...      ...      ...      ...         ...  
16240      Racing             Unknown     0.01        0        0           0  
16275     Shooter               Atari     2.36     0.14       0M        0.03  
16278      Sports             Unknown      0.7        0        0         50K  
16425        Misc             Unknown     0.06        0        0           0  
16432      Puzzle             Unknown     0.13        0        0        0.01  

[271 rows x 10 columns]

In [75]:
#'Genre'(게임장르)는 게임이름으로 유추가능하고,
#'Publisher'(게임회사이름)은 없어도 연도별 출고량이나 게임장르를 알 수 있어서 유의미한 데이터라고 판단해서 삭제 안함. 
df1['Genre']|df1['Publisher'].fillna(0, inplace=True)

0        False
1        False
2        False
3        False
4        False
         ...  
16593    False
16594    False
16595    False
16596    False
16597    False
Name: Genre, Length: 16598, dtype: bool

In [76]:
# column명 'Genre','Publisher' 결측치를 0으로 채우고 난 후 결측치 확인 - df1
pd.DataFrame(df1.isnull().sum(), columns=["결측치 개수"])

결측치 개수
Unnamed: 0        0
Name              0
Platform          0
Year            271
Genre             0
Publisher         0
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0

In [77]:
# 결측데이터를 삭제한 이유는 다음분기 게임 설계를 위해 
#연도별 게임트렌드를 알아야 하는데
#삭제하지 않고 값을 0으로 두면 정확한 연도별 게임트렌드 분석수치가 달라질것 같아서 삭제하기로 결정했고,
#전체 데이터 16597개 중 Year 결측치271개는 1.63%정도여서, 삭제한다해도 분석결과에 큰 영향을 주지 않을거라고 생각한다. 

df1.dropna(axis=0, inplace=True)
df1.shape

(16327, 10)

In [78]:
# column_Year을 이해하기 쉽게 int로 변환
#전체 데이터 보니깐 년도만 표시해서 float형태로 있을 필요가 없음
df1 = df1.astype({'Year':'int'})

In [79]:
#제대로 삭제 되었는지 다시 확인
pd.DataFrame(df1.isnull().sum(), columns=["결측치 개수"])

결측치 개수
Unnamed: 0        0
Name              0
Platform          0
Year              0
Genre             0
Publisher         0
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0

In [45]:
df1.sample(5)

Unnamed: 0                                    Name Platform  Year  \
3377         3378             Blade Arcus from Shining EX      PS3  2015   
1035         1036  The Legend of Zelda: Twilight Princess      Wii  2006   
11197       11198                                BoomBots       PS  1999   
441           442                        Irritating Stick       PS  1998   
6793         6794                        Reality Fighters      PSV  2012   

          Genre                    Publisher NA_Sales EU_Sales JP_Sales  \
3377   Fighting                         Sega        0        0    0.02M   
1035     Action                     Nintendo     3.83     2.19      0.6   
11197  Fighting              SouthPeak Games     0.06     0.04        0   
441      Action                       Saurus     0.01     0.01        0   
6793   Fighting  Sony Computer Entertainment     0.06     0.06        0   

      Other_Sales  
3377            0  
1035          0.7  
11197        0.01  
441             0  
6793         0.03

In [82]:
# 데이터셋의 중복된 데이터 개수 확인
df1.duplicated().sum()

0